In [1]:
import subprocess
import json

In [2]:
def regex_equiv(gold, predicted):
    if gold == predicted:
        return True
    try:
        out = subprocess.check_output(['java', '-jar', 'regex_dfa_equals.jar', '{}'.format(gold), '{}'.format(predicted)])
#          print("out: {}".format(out))
        #print(out.decode('utf-8'))
        if '\n1' in out.decode('utf-8'):
            return True
        else:
            return False
    except Exception as e:
        return False


In [ ]:

names = ["Phi_Raw_Output","Phi_Refined_Output"]
for name in names:
    print("Starting: ", name)
    filename = f'../Generation/{name}.json'
    outfilename = f'./{name}_DFA_Result.json'

    with open(filename, 'r') as f:
        data = json.load(f)
    print(len(data))

    dfa_result = []
    for item in data:
        print("Starting item: ", item['id'])
        id = item['id']
        gold_expression = item['expression']
        # print ("Expression: ", gold_expression)
        for i in range(len(item['phi_output'])):
            predicted_expression = item['phi_output'][i]['text']
            item['completion'] = predicted_expression
            passed = regex_equiv(gold_expression, predicted_expression)
            
            dfa_result.append({
                'id': id,
                'completion': predicted_expression,
                'passed': passed
            })  
        print("Ending item: ", item['id'])

    with open(outfilename, 'w') as f:
        json.dump(dfa_result, f, indent=4)